In [1]:
# this is the testing area

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from stn import spatial_transformer_network as transformer
from tensorflow.keras.regularizers import l2
import tensorflow.keras.activations as ac
import numpy as np
import matplotlib.pyplot as plt
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import cv2



In [3]:
train = ImageDataGenerator(rescale=1 / 255,validation_split=0.2)
test = ImageDataGenerator(rescale=1 / 255)

In [4]:
image_size=(250,250)

In [5]:
train_dataset = train.flow_from_directory("../database/sharpness1reduce/train",
                                          target_size=image_size,
                                          class_mode="categorical",
                                          subset="training",
                                          
                                         )
valid_dataset = train.flow_from_directory("../database/sharpness1reduce/train",
                                          target_size=image_size,
                                          class_mode="categorical",                                          
                                          subset="validation")

test_dataset = test.flow_from_directory("../database/sharpness1reduce/test",
                                        target_size=image_size,                                       
                                        class_mode="categorical")

Found 3282 images belonging to 49 classes.
Found 795 images belonging to 49 classes.
Found 1919 images belonging to 49 classes.


In [6]:
model = tf.keras.models.Sequential()
    
model.add(tf.keras.Input(shape=(250, 250, 3)))
model.add(tf.keras.layers.Conv2D(32,(7, 7),padding="same",activation="relu"))       
model.add(tf.keras.layers.Conv2D(32,(5, 5),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(32,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2 ,2)))
        
model.add(tf.keras.layers.Conv2D(64,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(64,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(64,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2 ,2)))
        
model.add(tf.keras.layers.Conv2D(128,(3, 3),padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(128,(3, 3),padding="same",activation="relu")) 
model.add(tf.keras.layers.Conv2D(128,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(256,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(256,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(256,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2 ,2)))

model.add(tf.keras.layers.Conv2D(512,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(512,(3, 3), padding="same",activation="relu"))
model.add(tf.keras.layers.Conv2D(512,(3, 3),activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2 ,2)))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(4096,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(2048,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(1024,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization()),
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(512,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
        
model.add(tf.keras.layers.Dense(256,kernel_regularizer=l2(0.005),bias_regularizer=l2(0.005)))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
        
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))


model.add(tf.keras.layers.Dense(49, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 250, 250, 32)      4736      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 250, 250, 32)      25632     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 248, 248, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 124, 124, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 122, 122, 64)      3

In [8]:

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_dataset, steps_per_epoch=40,validation_data=valid_dataset, epochs=500,shuffle=True)


score = model.evaluate(test_dataset, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])
print("echo this is the end ***")


  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 40 steps, validate for 25 steps
Epoch 1/500
40/40 [==============================] - 13s 335ms/step - loss: 19.8008 - accuracy: 0.1063 - val_loss: 17.8432 - val_accuracy: 0.0403
Epoch 2/500
40/40 [==============================] - 11s 268ms/step - loss: 15.0670 - accuracy: 0.1195 - val_loss: 25.3820 - val_accuracy: 0.1119
Epoch 3/500
40/40 [==============================] - 11s 265ms/step - loss: 12.3425 - accuracy: 0.1351 - val_loss: 16.9948 - val_accuracy: 0.1560
Epoch 4/500
40/40 [==============================] - 11s 263ms/step - loss: 10.5364 - accuracy: 0.1825 - val_loss: 12.0813 - val_accuracy: 0.1396
Epoch 5/500
40/40 [==============================] - 11s 264ms/step - loss: 9.4728 - accuracy: 0.1998 - val_loss: 8.7745 - val_accuracy: 0.2264
Epoch 6/500
40/40 [==============================] - 11s 263ms/step - loss: 8.4109 - accuracy: 0.2283 - val_loss: 213.7127 - val_accuracy: 0.0403
Epoch 7/500
40/40 [===============

Epoch 56/500
40/40 [==============================] - 11s 265ms/step - loss: 4.6554 - accuracy: 0.5300 - val_loss: 5.1675 - val_accuracy: 0.3610
Epoch 57/500
40/40 [==============================] - 11s 264ms/step - loss: 4.5498 - accuracy: 0.5158 - val_loss: 4.1518 - val_accuracy: 0.6465
Epoch 58/500
40/40 [==============================] - 11s 265ms/step - loss: 4.3655 - accuracy: 0.5269 - val_loss: 4.9291 - val_accuracy: 0.4176
Epoch 59/500
40/40 [==============================] - 11s 265ms/step - loss: 4.5051 - accuracy: 0.5205 - val_loss: 4.1991 - val_accuracy: 0.6226
Epoch 60/500
40/40 [==============================] - 11s 265ms/step - loss: 4.4438 - accuracy: 0.5569 - val_loss: 4.7312 - val_accuracy: 0.4981
Epoch 61/500
40/40 [==============================] - 11s 267ms/step - loss: 4.7820 - accuracy: 0.5148 - val_loss: 4.8669 - val_accuracy: 0.5648
Epoch 62/500
40/40 [==============================] - 11s 267ms/step - loss: 4.8740 - accuracy: 0.5164 - val_loss: 6.9379 - val_ac

40/40 [==============================] - 11s 266ms/step - loss: 4.0129 - accuracy: 0.6959 - val_loss: 3.5082 - val_accuracy: 0.7912
Epoch 113/500
40/40 [==============================] - 11s 268ms/step - loss: 4.0873 - accuracy: 0.6680 - val_loss: 7.8860 - val_accuracy: 0.3308
Epoch 114/500
40/40 [==============================] - 11s 266ms/step - loss: 4.6449 - accuracy: 0.6864 - val_loss: 3.9905 - val_accuracy: 0.8038
Epoch 115/500
40/40 [==============================] - 11s 268ms/step - loss: 4.0900 - accuracy: 0.6914 - val_loss: 3.5732 - val_accuracy: 0.7887
Epoch 116/500
40/40 [==============================] - 11s 267ms/step - loss: 4.0217 - accuracy: 0.6906 - val_loss: 3.9951 - val_accuracy: 0.7862
Epoch 117/500
40/40 [==============================] - 11s 266ms/step - loss: 4.2356 - accuracy: 0.6833 - val_loss: 3.7293 - val_accuracy: 0.8038
Epoch 118/500
40/40 [==============================] - 11s 266ms/step - loss: 4.1629 - accuracy: 0.6856 - val_loss: 7.4109 - val_accuracy:

40/40 [==============================] - 11s 266ms/step - loss: 3.7872 - accuracy: 0.7417 - val_loss: 45.7293 - val_accuracy: 0.7094
Epoch 169/500
40/40 [==============================] - 11s 268ms/step - loss: 4.0303 - accuracy: 0.7469 - val_loss: 3.6281 - val_accuracy: 0.8465
Epoch 170/500
40/40 [==============================] - 11s 266ms/step - loss: 3.6445 - accuracy: 0.7615 - val_loss: 3.3096 - val_accuracy: 0.8314
Epoch 171/500
40/40 [==============================] - 11s 266ms/step - loss: 3.9797 - accuracy: 0.7283 - val_loss: 21.2332 - val_accuracy: 0.8314
Epoch 172/500
40/40 [==============================] - 11s 268ms/step - loss: 4.1461 - accuracy: 0.7328 - val_loss: 3.7922 - val_accuracy: 0.8088
Epoch 173/500
40/40 [==============================] - 11s 266ms/step - loss: 3.9876 - accuracy: 0.7314 - val_loss: 118.8650 - val_accuracy: 0.3962
Epoch 174/500
40/40 [==============================] - 11s 268ms/step - loss: 3.6365 - accuracy: 0.7578 - val_loss: 3.3542 - val_accur

Epoch 224/500
40/40 [==============================] - 11s 269ms/step - loss: 3.3193 - accuracy: 0.8039 - val_loss: 3.4780 - val_accuracy: 0.8830
Epoch 225/500
40/40 [==============================] - 11s 268ms/step - loss: 3.4831 - accuracy: 0.7828 - val_loss: 3.0008 - val_accuracy: 0.8881
Epoch 226/500
40/40 [==============================] - 11s 267ms/step - loss: 3.4816 - accuracy: 0.7709 - val_loss: 221.7447 - val_accuracy: 0.8830
Epoch 227/500
40/40 [==============================] - 11s 266ms/step - loss: 3.7455 - accuracy: 0.7923 - val_loss: 19.6565 - val_accuracy: 0.6516
Epoch 228/500
40/40 [==============================] - 11s 266ms/step - loss: 3.7194 - accuracy: 0.7986 - val_loss: 3.6002 - val_accuracy: 0.8302
Epoch 229/500
40/40 [==============================] - 11s 268ms/step - loss: 3.9787 - accuracy: 0.7719 - val_loss: 4.4060 - val_accuracy: 0.8528
Epoch 230/500
40/40 [==============================] - 11s 268ms/step - loss: 3.7342 - accuracy: 0.7836 - val_loss: 4.189

40/40 [==============================] - 11s 266ms/step - loss: 3.7243 - accuracy: 0.8207 - val_loss: 3.3503 - val_accuracy: 0.8792
Epoch 281/500
40/40 [==============================] - 11s 268ms/step - loss: 3.5271 - accuracy: 0.8141 - val_loss: 3.1893 - val_accuracy: 0.9094
Epoch 282/500
40/40 [==============================] - 11s 266ms/step - loss: 3.6621 - accuracy: 0.8025 - val_loss: 3.5288 - val_accuracy: 0.8818
Epoch 283/500
40/40 [==============================] - 11s 266ms/step - loss: 3.6462 - accuracy: 0.8412 - val_loss: 3.1559 - val_accuracy: 0.9170
Epoch 284/500
40/40 [==============================] - 11s 268ms/step - loss: 3.2634 - accuracy: 0.8352 - val_loss: 2.9675 - val_accuracy: 0.8906
Epoch 285/500
40/40 [==============================] - 11s 266ms/step - loss: 3.5675 - accuracy: 0.8081 - val_loss: 3.4311 - val_accuracy: 0.8881
Epoch 286/500
40/40 [==============================] - 11s 267ms/step - loss: 3.6354 - accuracy: 0.8152 - val_loss: 3.2539 - val_accuracy:

40/40 [==============================] - 11s 268ms/step - loss: 3.7660 - accuracy: 0.8125 - val_loss: 4.3865 - val_accuracy: 0.7597
Epoch 337/500
40/40 [==============================] - 11s 268ms/step - loss: 3.2650 - accuracy: 0.8484 - val_loss: 3.3108 - val_accuracy: 0.9421
Epoch 338/500
40/40 [==============================] - 11s 268ms/step - loss: 3.0180 - accuracy: 0.8508 - val_loss: 2.8617 - val_accuracy: 0.9245
Epoch 339/500
40/40 [==============================] - 11s 266ms/step - loss: 2.9174 - accuracy: 0.8507 - val_loss: 2.6879 - val_accuracy: 0.9283
Epoch 340/500
40/40 [==============================] - 11s 268ms/step - loss: 3.0755 - accuracy: 0.8438 - val_loss: 3.0308 - val_accuracy: 0.9283
Epoch 341/500
40/40 [==============================] - 11s 268ms/step - loss: 3.4602 - accuracy: 0.8242 - val_loss: 3.4038 - val_accuracy: 0.8981
Epoch 342/500
40/40 [==============================] - 11s 268ms/step - loss: 3.4488 - accuracy: 0.8555 - val_loss: 3.0632 - val_accuracy:

40/40 [==============================] - 11s 265ms/step - loss: 3.1164 - accuracy: 0.8547 - val_loss: 2.7743 - val_accuracy: 0.9509
Epoch 393/500
40/40 [==============================] - 11s 267ms/step - loss: 3.4525 - accuracy: 0.8258 - val_loss: 3.5036 - val_accuracy: 0.9170
Epoch 394/500
40/40 [==============================] - 11s 267ms/step - loss: 3.5340 - accuracy: 0.8539 - val_loss: 3.0213 - val_accuracy: 0.9384
Epoch 395/500
40/40 [==============================] - 11s 267ms/step - loss: 3.1804 - accuracy: 0.8523 - val_loss: 2.8123 - val_accuracy: 0.9409
Epoch 396/500
40/40 [==============================] - 11s 267ms/step - loss: 2.9938 - accuracy: 0.8805 - val_loss: 2.5796 - val_accuracy: 0.9396
Epoch 397/500
40/40 [==============================] - 11s 267ms/step - loss: 2.8940 - accuracy: 0.8547 - val_loss: 2.8638 - val_accuracy: 0.9107
Epoch 398/500
40/40 [==============================] - 11s 267ms/step - loss: 3.2894 - accuracy: 0.8289 - val_loss: 3.0534 - val_accuracy:

40/40 [==============================] - 11s 265ms/step - loss: 2.8542 - accuracy: 0.8823 - val_loss: 2.5415 - val_accuracy: 0.9208
Epoch 449/500
40/40 [==============================] - 11s 265ms/step - loss: 2.8115 - accuracy: 0.8555 - val_loss: 2.8389 - val_accuracy: 0.9409
Epoch 450/500
40/40 [==============================] - 11s 266ms/step - loss: 3.5293 - accuracy: 0.8389 - val_loss: 3.2292 - val_accuracy: 0.9396
Epoch 451/500
40/40 [==============================] - 11s 265ms/step - loss: 3.3382 - accuracy: 0.8570 - val_loss: 2.8736 - val_accuracy: 0.9535
Epoch 452/500
40/40 [==============================] - 11s 268ms/step - loss: 3.2382 - accuracy: 0.8602 - val_loss: 2.8931 - val_accuracy: 0.9358
Epoch 453/500
40/40 [==============================] - 11s 265ms/step - loss: 3.2812 - accuracy: 0.8404 - val_loss: 8.3893 - val_accuracy: 0.8704
Epoch 454/500
40/40 [==============================] - 11s 267ms/step - loss: 3.1916 - accuracy: 0.8586 - val_loss: 3.0821 - val_accuracy: